In [1]:
import pandas as pd
#constructor_results = pd.read_csv("constructor_results.csv")
#constructor_standings = pd.read_csv("constructor_standings.csv")
#constructors = pd.read_csv('constructors.csv')
#driver_standings = pd.read_csv("driver_standings.csv")
#pit_stops = pd.read_csv("pit_stops.csv")
#qualifying = pd.read_csv("qualifying.csv")
#results = pd.read_csv("results.csv")
#seasons = pd.read_csv("seasons.csv")
#status = pd.read_csv("status.csv")

drivers = pd.read_csv('driver.csv')
circuits = pd.read_csv("circuits.csv")
races = pd.read_csv("races.csv")
lap_times = pd.read_csv("lap_times.csv")

In [2]:
# drivers subset for join 
races_subset = races[["raceId", "circuitId","name", "date"]]

In [3]:
# drivers subset
drivers["fullname"] = drivers["forename"].map(str) + " " + drivers["surname"]
drivers_subset = drivers[["driverId", "fullname"]]

In [4]:
# rename some columns
circuits = circuits.rename(columns={'circuitID': 'circuitId', "name":"circuit_name"})

In [5]:
# subset to maintain intitial data
lap_times_subset = lap_times

# join driver to driver id
lap_times_subset = pd.merge(left = lap_times_subset, right = drivers_subset, on = "driverId")

# join raceId
lap_times_subset = pd.merge(left = lap_times_subset, right = races_subset, on = "raceId")

# join by circuit
lap_times_subset = pd.merge(left = circuits[["circuitId", "circuit_name"]], right = lap_times_subset, on = "circuitId")

In [7]:
# convert to datetime to make date manipulation easier
lap_times_subset['date'] = pd.to_datetime(lap_times_subset['date'])
# make year column
lap_times_subset["year"] = lap_times_subset.date.dt.year

In [14]:
# make dataframe to be plotted
mean_laptimes_by_track = lap_times_subset[["year", "circuit_name", "milliseconds"]]
# change milliseconds name to time
mean_laptimes_by_track = mean_laptimes_by_track.rename(columns = {"milliseconds":"time"})

In [15]:
# group by circuit and year and find average time
mean_laptimes_by_track = mean_laptimes_by_track.groupby(["circuit_name", "year"]).mean()

# convert ms to time
mean_laptimes_by_track.time = pd.to_timedelta(mean_laptimes_by_track.time, unit='ms')

# view
mean_laptimes_by_track

time
circuit_name                   year                
A1-Ring                        1997 00:01:14.986253
                               1998 00:01:18.521027
                               1999 00:01:16.098619
                               2000 00:01:16.906979
                               2001 00:01:16.860576
                               2002 00:01:20.723668
                               2003 00:01:14.695398
Albert Park Grand Prix Circuit 1996 00:01:39.021050
                               1997 00:01:36.326919
                               1998 00:01:37.903070
                               1999 00:01:42.112660
                               2000 00:01:39.751705
                               2001 00:01:44.914228
                               2002 00:01:47.321217
                               2003 00:01:39.997816
                               2004 00:01:30.808457
                               2005 00:01:30.332472
                               2006 00:01:41.650961
                               2007 00:01:30.397869
                               2008 00:01:38.557182
                               2009 00:01:37.427745
                               2010 00:01:41.076809
                               2011 00:01:35.618773
                               2012 00:01:38.469929
                               2013 00:01:34.959435
                               2014 00:01:40.366244
                               2015 00:01:37.062155
                               2016 00:01:55.614283
                               2017 00:01:31.105888
                               2018 00:01:33.258057
...                                             ...
Suzuka Circuit                 2000 00:01:44.520163
                               2001 00:01:41.900901
                               2002 00:01:40.953986
                               2003 00:01:38.206530
                               2004 00:01:37.773037
                               2005 00:01:43.227445
                               2006 00:01:37.062217
                               2009 00:01:40.347669
                               2010 00:01:44.808702
                               2011 00:01:44.489398
                               2012 00:01:42.524973
                               2013 00:01:40.004580
                               2014 00:02:35.868392
                               2015 00:01:41.991428
                               2016 00:01:39.930429
                               2017 00:01:40.707127
Valencia Street Circuit        2008 00:01:42.130726
                               2009 00:01:42.151677
                               2010 00:01:47.433160
                               2011 00:01:47.504345
                               2012 00:01:50.761681
Yas Marina Circuit             2009 00:01:43.623088
                               2010 00:01:50.506406
                               2011 00:01:48.615136
                               2012 00:01:56.403745
                               2013 00:01:48.937909
                               2014 00:01:49.714986
                               2015 00:01:49.684115
                               2016 00:01:48.479962
                               2017 00:01:44.873025

[410 rows x 1 columns]